In [31]:
from credentials import user, password, rds_host
import pymysql
import pandas as pd
from boston_functions import *
import re
import numpy as np
from liquid import liquids
from garnish import garnishes
import random
from fractions import Fraction

In [3]:
data = pd.read_csv("./mr-boston-all-glasses.csv")

data = data[data.loc[:, "glass-size"].notna()]
# data

In [5]:
valid_units = ["oz", "tsp", "splash", "dash"]

fill_liquid = ["ginger ale", "carbonated water", "cola", "water", "chilled champagne", "soda water", 
               "club soda", "ginger ale or soda water", "lemon-lime soda", "ginger beer", "bitter lemon soda",
               "apple juice", "orange juice"]

invaild_ingredients = ['chopped', 'cut in half', 'cut in halves', 'cut into halves', 'flamed', 'hulled', 'long', 'skinned',
                       'whipped', "preferably b.a. reynold's", 'preferably jamaican', 'preferably japanese', 
                       'preferably pedro ximenez', "such as bittermen's elemakule", 'such as demerara', 
                       'such as islay or skye', 'such as nasturtium']

ingredient_indicies = range(3, 14)

In [4]:
categories = list(set(data.iloc[:, 1])) + ["AI Instant Classic"]

In [6]:
all_recipies, measures = get_cocktail_recipies(data, ingredient_indicies, liquids, garnishes, invaild_ingredients, valid_units)

In [25]:
all_recipies[1]

{'name': 'Fort Lauderdale',
 'category': 'Cocktail Classics',
 'instructions': 'Shake with ice and strain into old-fashioned glass over ice cubes. Add a slice of orange.',
 'glass': 'Old-Fashioned Glass',
 'glass_size': 6,
 'recipe': [['1 1/2 oz', 'batavia arrack or light rum'],
  ['1/2 oz', 'sweet vermouth'],
  ['1/4 oz', 'each orange juice and sweet sherry'],
  ['1/4 oz', "rose's lime juice"]]}

In [8]:
for i in range(10):
    print(all_recipies[i]["recipe"])

[['2 oz', 'batavia arrack or light rum'], ['1 oz', 'passion fruit syrup'], ['1 oz', 'fresh lemon juice'], ['1 oz', "rose's lime juice"]]
[['1 1/2 oz', 'batavia arrack or light rum'], ['1/2 oz', 'sweet vermouth'], ['1/4 oz', 'each orange juice and sweet sherry'], ['1/4 oz', "rose's lime juice"]]
[['3 oz', 'apple schnapps'], ['1 oz', 'cinnamon schnapps'], ['add', 'small pineapple slice']]
[['1/2 oz', "rose's lime juice"], ['add', 'powdered sugar'], ['2 oz', 'batavia arrack or light rum']]
[['1 1/2 oz', 'dark rum'], ['2 oz', 'each cranberry juice and orange juice'], ['2 oz', 'pineapple juice'], ['1 oz', 'orange curacao'], ['1 oz', 'sour mix']]
[['2 oz', 'old kentucky tavern bourbon whiskey'], ['1 oz', 'fresh lemon juice'], ['1/2 oz', 'simple syrup'], ['1 oz', 'ginger ale or soda water'], ['add', 'orange and lemon wheels'], ['add', 'pineapple wedge skewered with maraschino cherry']]
[['1 1/4 oz', 'batavia arrack or light rum'], ['1 1/2 oz', 'wild cherry flavored brandy'], ['1 oz', 'light c

In [8]:
#set empty lists for cocktail names, glass names, and glass sizes
names = []
glasses = []
glass_sizes = []

# locate cocktail names, glass names, and glass sizes
for i in range(len(all_recipies)):
    names.append(all_recipies[i]["name"])
    glasses.append(all_recipies[i]["glass"])
    glass_sizes.append(all_recipies[i]["glass_size"])
#create df
glass_df = pd.DataFrame({"cocktail_name": names, "glass_name": glasses, "glass_size": glass_sizes})

# glass_df.head()

In [9]:
#read glass svg csv
svg_data = pd.read_csv("../glasses.csv")
svg_data.head()

,mask,maskHeight,maskTopMargin,name,path
0,"m 43.972082,0.754995 c -0.0081,0.25817 -0.0643...",28.0,1.0,COCKTAIL_1,"m 18.233415,27.465585 c -1.678156,-1.38771 -2...."
1,"M 20.289051,33.296931 C 19.290178,31.958842 18...",33.0,1.5,champagne_flute,"M 20.229203,33.296931 C 19.23033,31.958842 18...."
2,"m 41.615,1.72 c 0,0 -1.500626,2.4979518 -2.263...",24.0,1.5,cocktail_sm_martini,"m 18.029995,23.735654 c -0.785523,-0.535326 -1..."
3,"m 11.238,1.6429413 c 0,0 0.843577,2.9274173 0....",33.5,1.5,poco_grande,"M 17.33604,34.049118 C 15.163032,33.299665 12...."
4,"m 41.544,16.418 c 0,0 -0.01131,3.428254 -0.003...",37.5,16.5,ROCK,"M 4.158169,53.270076 C 3.5567763,52.526162 3.6..."


In [10]:
# df = svg_data.loc[svg_data["name"]=="fluted_lager", :]
# df['mask'].to_string(index=False)

In [11]:
#set empty lists for svg data
masks = []
paths = []
mask_heights = []
mask_top_margins = []

#match mr. boston  db glass names with svg glass names
for i in range(len(glass_df)):
    glass_name = glass_df.iloc[i, 1]
    if glass_name == "Beer Mug":
        df = svg_data.loc[svg_data["name"]=="fluted_lager", :]
        masks.append(df["mask"].to_string(index=False))
        paths.append(df["path"].to_string(index=False))
        mask_heights.append(df["maskHeight"].to_string(index=False))
        mask_top_margins.append(df["maskTopMargin"].to_string(index=False))
    elif (glass_name == "Brandy Snifter") or (glass_name == "Red-Wine Glass") or (glass_name == "White-Wine Glass"):
        df = svg_data.loc[svg_data["name"]=="poco_grande", :]
        masks.append(df["mask"].to_string(index=False))
        paths.append(df["path"].to_string(index=False))
        mask_heights.append(df["maskHeight"].to_string(index=False))
        mask_top_margins.append(df["maskTopMargin"].to_string(index=False))
    elif glass_name == "Champagne Flute":
        df = svg_data.loc[svg_data["name"]=="champagne_extra_fluted", :]
        masks.append(df["mask"].to_string(index=False))
        paths.append(df["path"].to_string(index=False))
        mask_heights.append(df["maskHeight"].to_string(index=False))
        mask_top_margins.append(df["maskTopMargin"].to_string(index=False))
    elif glass_name == "Cocktail Glass":
        df = svg_data.loc[svg_data["name"]=="cocktail_lg", :]
        masks.append(df["mask"].to_string(index=False))
        paths.append(df["path"].to_string(index=False))
        mask_heights.append(df["maskHeight"].to_string(index=False))
        mask_top_margins.append(df["maskTopMargin"].to_string(index=False))
    elif glass_name == "Collins Glass":
        df = svg_data.loc[svg_data["name"]=="collins", :]
        masks.append(df["mask"].to_string(index=False))
        paths.append(df["path"].to_string(index=False))
        mask_heights.append(df["maskHeight"].to_string(index=False))
        mask_top_margins.append(df["maskTopMargin"].to_string(index=False))
    elif (glass_name == "Cordial or Pony Glass") or (glass_name == "Pousse-Cafe Glass") or (glass_name == "Sherry Glass"):
        df = svg_data.loc[svg_data["name"]=="pousse_cafe", :]
        masks.append(df["mask"].to_string(index=False))
        paths.append(df["path"].to_string(index=False))
        mask_heights.append(df["maskHeight"].to_string(index=False))
        mask_top_margins.append(df["maskTopMargin"].to_string(index=False))
    elif glass_name == "Highball Glass":
        df = svg_data.loc[svg_data["name"]=="highball", :]
        masks.append(df["mask"].to_string(index=False))
        paths.append(df["path"].to_string(index=False))
        mask_heights.append(df["maskHeight"].to_string(index=False))
        mask_top_margins.append(df["maskTopMargin"].to_string(index=False))
    elif glass_name == "Hurricane Glass":
        df = svg_data.loc[svg_data["name"]=="hurricane", :]
        masks.append(df["mask"].to_string(index=False))
        paths.append(df["path"].to_string(index=False))
        mask_heights.append(df["maskHeight"].to_string(index=False))
        mask_top_margins.append(df["maskTopMargin"].to_string(index=False))
    elif glass_name == "Irish Coffee Glass":
        df = svg_data.loc[svg_data["name"]=="irish_coffee", :]
        masks.append(df["mask"].to_string(index=False))
        paths.append(df["path"].to_string(index=False))
        mask_heights.append(df["maskHeight"].to_string(index=False))
        mask_top_margins.append(df["maskTopMargin"].to_string(index=False))
    elif glass_name == "Old-Fashioned Glass":
        df = svg_data.loc[svg_data["name"]=="old_fashioned", :]
        masks.append(df["mask"].to_string(index=False))
        paths.append(df["path"].to_string(index=False))
        mask_heights.append(df["maskHeight"].to_string(index=False))
        mask_top_margins.append(df["maskTopMargin"].to_string(index=False))
    elif (glass_name == "Punch Cup") or (glass_name == "Shot Glass"):
        df = svg_data.loc[svg_data["name"]=="rocks", :]
        masks.append(df["mask"].to_string(index=False))
        paths.append(df["path"].to_string(index=False))
        mask_heights.append(df["maskHeight"].to_string(index=False))
        mask_top_margins.append(df["maskTopMargin"].to_string(index=False))
    elif glass_name == "Sour Glass":
        df = svg_data.loc[svg_data["name"]=="champagne_flute", :]
        masks.append(df["mask"].to_string(index=False))
        paths.append(df["path"].to_string(index=False))
        mask_heights.append(df["maskHeight"].to_string(index=False))
        mask_top_margins.append(df["maskTopMargin"].to_string(index=False))
# print(len(masks), len(paths), len(mask_heights), len(mask_top_margins))

#add svg info to glass df
glass_df["mask"] = masks
glass_df["path"] = paths
glass_df["mask_height"] = mask_heights
glass_df["mask_top_margin"] = mask_top_margins
glass_df.head()

In [12]:
def populate_glass_table(glass_df):
    conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
    cursor = conn.cursor()
    cursor.execute('USE cocktailproject')
    for row in range(len(glass_df)):
        glass_name = glass_df.iloc[row, 1]
        glass_size = glass_df.iloc[row, 2]
        mask = glass_df.iloc[row, 3]
        path = glass_df.iloc[row, 4]
        mask_height = glass_df.iloc[row, 5]
        mask_top_margin = glass_df.iloc[row, 6]
        print(glass_name, glass_size)
        sql = f"INSERT INTO Glasses (Glass_Name, Glass_Size, Mask, Path, Mask_Height, Mask_Top_Margin) VALUES ('{glass_name}', '{glass_size}', '{mask}', '{path}', '{mask_height}', '{mask_top_margin}');"
        cursor.execute(sql)
    conn.commit()
    cursor.execute("SELECT * FROM Glasses")
    data = cursor.fetchall()
    print(data)
    conn.close()
# populate_glass_table(glass_df)

In [13]:
cocktail_table_sql = "CREATE TABLE Cocktails ( \
                        Cocktail_ID INT NOT NULL PRIMARY KEY AUTO_INCREMENT \
                        , Cocktail_Name VARCHAR(80) \
                        , Glass_ID INT \
                        , Category_ID INT \
                        , FOREIGN KEY (Glass_ID) REFERENCES Glasses(Glass_ID) \
                        , FOREIGN KEY (Category_ID) REFERENCES Categories(Category_ID) \
                        );"

In [17]:
#set empty lists for category and glass ids
category_ids = []
glass_ids = []

#connect to sql
conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
#create cursor object
cursor = conn.cursor()
#use cocktail db
cursor.execute('USE cocktailproject')

#for each recipe,
for i in range(len(all_recipies)):
    #find cocktail category
    category = all_recipies[i]["category"]
    
    #select category id and append to category id list
    sql = f"SELECT Category_ID FROM Categories WHERE Category_Name='{category}'"
    cursor.execute(sql)
    data = cursor.fetchall()
    category_ids.append(data[0][0])
    
    #select glass id and append to glass id list
    sql = f"SELECT Glass_ID FROM Glasses WHERE Glass_Name='{glasses[i]}'"
    cursor.execute(sql)
    data = cursor.fetchall()
    glass_ids.append(data[0][0])

#close sql connection
conn.close()
# print(category_ids[0], glass_ids[0])

In [18]:
#set empty lists for instructions
instructions_list = []
instr_lengths = []
#for all recipes,
for i in range(len(all_recipies)):
    #find instructions and append to instructions list
    instructions = all_recipies[i]["instructions"]
    instructions_list.append(instructions)
    instr_lengths.append(len(instructions))

In [19]:
#create cocktail df
cocktail_df = pd.DataFrame({"cocktail_name": names, "glass_id": glass_ids, "category_ids": category_ids, "instructions": instructions_list})
cocktail_df.head()

,cocktail_name,glass_id,category_ids,instructions
0,Gauguin,1,6,Combine ingredients with a cup of crushed ice ...
1,Fort Lauderdale,1,6,Shake with ice and strain into old-fashioned g...
2,Apple Pie,1,1,Pour into ice-filled old-fashioned glass. Garn...
3,Cuban Cocktail No. 1,4,6,Shake with ice and strain into cocktail glass.
4,Cool Carlos,5,6,"Mix all ingredients except curacao with ice, s..."


In [20]:
def populate_cocktail_table(cocktail_df):
    conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
    cursor = conn.cursor()
    cursor.execute('USE cocktailproject')
    for row in range(len(cocktail_df)):
        cocktail_name = cocktail_df.iloc[row, 0]
        glass_id = cocktail_df.iloc[row, 1]
        category_id = cocktail_df.iloc[row, 2]
        instructions = cocktail_df.iloc[row, 3]
        print(cocktail_name)
        sql = f"INSERT INTO Cocktails (Cocktail_Name, Glass_ID, Category_ID, Instructions) VALUES ('{cocktail_name}', '{glass_id}', '{category_id}', '{instructions}');"
        cursor.execute(sql)
    conn.commit()
    cursor.execute("SELECT * FROM Cocktails")
    data = cursor.fetchall()
    print(data)
    conn.close()
# populate_cocktail_table(cocktail_df)

In [21]:
liquid_instructions_table_sql = "CREATE TABLE Liquid_Instuctions ( \
                                Liquid_Instruction_ID INT NOT NULL PRIMARY KEY AUTO_INCREMENT \
                                , Cocktail_ID INT \
                                , Liquid_ID INT \
                                , Measure VARCHAR(24) \
                                , FOREIGN KEY (Cocktail_ID) REFERENCES Cocktails(Cocktail_ID) \
                                , FOREIGN KEY (Liquid_ID) REFERENCES Liquids(Liquid_ID) \
                                );"

In [30]:
for i in range(len(all_recipies)):
    if all_recipies[i]["name"] == "Tequila Sunrise":
        print(all_recipies[i]["instructions"])
        instructions = re.sub(r'["]', '\'', all_recipies[i]["instructions"])
        print(instructions)

Pour tequila and orange juice into ice-filled highball glass. Stir and add ice. Slowly pour in grenadine and allow to settle at glass bottom. Before drinking, stir to create the "sunrise"
Pour tequila and orange juice into ice-filled highball glass. Stir and add ice. Slowly pour in grenadine and allow to settle at glass bottom. Before drinking, stir to create the 'sunrise'


In [22]:
#set empty lists for cocktail ids, liquid ingredients and measures
# cocktail_ids_list = []
liquid_ingredient_list = []
liquid_measure_list = []
liquid_measure_float_list = []

#for each recipe,
for i in range(len(all_recipies)):
    #set cocktail_id
    cocktail_id = cocktail_id_list[i]
    #find ingredient list
    ingredient_list = all_recipies[i]["recipe"]
    #find total glass volume
    total_glass_volume = all_recipies[i]["glass_size"]
    #set total liquid measure to 0
    total_liquid_measure = 0
    
    #for each non-garnish ingredient,
    for measure, ingredient in ingredient_list:
        measure = measure.strip()
        if measure == "add":
            pass
        elif ingredient == "pimm's cup":#pimm's cup did not make it onto out liquids list
            pass
        
        else:
            #add to lists
            cocktail_ids_list.append(cocktail_id)
            liquid_measure_list.append(measure)
            liquid_ingredient_list.append(ingredient)
            
            #split measure on spaces
            split_measure = measure.split(" ")

            #if measure is compound fraction,
            if len(split_measure) > 2:
                #recombine compound fraction
                measure = split_measure[0]+" "+split_measure[1]
                #convert to float
                measure_float = float(sum(Fraction(s) for s in measure.split()))
                #add to float list
                liquid_measure_float_list.append(measure_float)
                #add measure to drink's total measure
                total_liquid_measure += measure_float
            #if measure is an integer,
            elif len(split_measure) > 1:
                #convert to float
                measure_float = float(sum(Fraction(s) for s in split_measure[0].split()))
                #if unit is smaller than oz, convert measure to oz
                if split_measure[1][:3] == "tsp":
                    measure_float = measure_float / 6
                elif split_measure[1][:4] == "dash":
                    measure_float = measure_float / 32
                elif split_measure[1][:6] == "splash":
                    measure_float = measure_float / 5
                #add to float list
                liquid_measure_float_list.append(measure_float)
                #add measure to drink's total measure
                total_liquid_measure += measure_float
            #if measure if fill,
            elif split_measure[0] == "fill":
                #calculate remaining volume
                remaining_volume = float(total_glass_volume) - float(total_liquid_measure)
                #set fill measure to half remaining volume
                measure_float = float(remaining_volume / 2)
                #add to float list
                liquid_measure_float_list.append(measure_float)
            
# #connect to sql
# conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
# #create cursor object
# cursor = conn.cursor()
# #use cocktail db
# cursor.execute('USE cocktailproject')

# #set empty list for liquid ids
# liquid_ids = []

# #for each ingredient,
# for ingredient in liquid_ingredient_list:
#     print(ingredient)
#     if ingredient[:6] == "rose's":
#         ingredient = ingredient[7:]
#         print(ingredient)
#         #select liquid id and append to liquid id list
#         sql = f'SELECT Liquid_ID FROM Liquids WHERE Liquid_Name="{ingredient}"'
#         cursor.execute(sql)
#         data = cursor.fetchall()
#         print(data[0][0])
#         liquid_ids.append(data[0][0])
#     elif ingredient[:8] =="peychaud":
#         ingredient = "peychauds bitters"
#         print(ingredient)
#         #select liquid id and append to liquid id list
#         sql = f'SELECT Liquid_ID FROM Liquids WHERE Liquid_Name="{ingredient}"'
#         cursor.execute(sql)
#         data = cursor.fetchall()
#         print(data[0][0])
#         liquid_ids.append(data[0][0])
#     else:
#         #select liquid id and append to liquid id list
#         sql = f'SELECT Liquid_ID FROM Liquids WHERE Liquid_Name="{ingredient}"'
#         cursor.execute(sql)
#         data = cursor.fetchall()
#         print(data[0][0])
#         liquid_ids.append(data[0][0])

# #close sql connection
# conn.close()
    
# liquid_ingredients_df = pd.DataFrame({"cocktail_id": cocktail_ids_list, "liquid_id": liquid_ids, 
#                                       "measure": liquid_measure_list})

# liquid_ingredients_df.head()

['2', 'oz']
2.0
['1', 'oz']
1.0
['1', 'oz']
1.0
['1', 'oz']
1.0
['1', '1/2', 'oz']
1.5
['1/2', 'oz']
0.5
['1/4', 'oz']
0.25
['1/4', 'oz']
0.25
['3', 'oz']
3.0
['1', 'oz']
1.0
['1/2', 'oz']
0.5
['2', 'oz']
2.0
['1', '1/2', 'oz']
1.5
['2', 'oz']
2.0
['2', 'oz']
2.0
['1', 'oz']
1.0
['1', 'oz']
1.0
['2', 'oz']
2.0
['1', 'oz']
1.0
['1/2', 'oz']
0.5
['1', 'oz']
1.0
['1', '1/4', 'oz']
1.25
['1', '1/2', 'oz']
1.5
['1', 'oz']
1.0
['2', 'oz']
2.0
['1', '1/2', 'oz']
1.5
['1', '1/2', 'oz']
1.5
['1', '1/2', 'oz']
1.5
['1/2', 'oz']
0.5
['1/2', 'oz']
0.5
['fill']
5.0
2.5
['fill']
5.0
2.5
['1', '1/2', 'oz']
1.5
['1/4', 'oz']
0.25
['1/4', 'oz']
0.25
['fill']
12.0
6.0
['1', '1/2', 'oz']
1.5
['1', 'oz']
1.0
['3/4', 'oz']
0.75
['1/2', 'oz']
0.5
['1', 'oz']
1.0
['1', '1/2', 'oz']
1.5
['1/2', 'oz']
0.5
['1/2', 'oz']
0.5
['3/4', 'oz']
0.75
['3/4', 'oz']
0.75
['1', 'oz']
1.0
['1/2', 'oz']
0.5
['1', 'oz']
1.0
['1/2', 'oz']
0.5
['1/2', 'oz']
0.5
['1/2', 'oz']
0.5
['1', 'oz']
1.0
['1', 'oz']
1.0
['1', 'oz']
1.0


In [53]:
print(len(cocktail_ids_list), len(liquid_ids), len(liquid_measure_list))

3323 3323 3323


In [ ]:
def populate_liquid_instructions_table(liquid_instructions_df):
    conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
    cursor = conn.cursor()
    cursor.execute('USE cocktailproject')
    for row in range(len(liquid_instructions_df)):
        cocktail_id = liquid_instructions_df.iloc[row, 0]
        liquid_id = liquid_instructions_df.iloc[row, 1]
        measure = liquid_instructions_df.iloc[row, 2]
        print(liquid_id, measure)
        sql = f"INSERT INTO Liquid_Instuctions (Cocktail_ID, Liquid_ID, Measure) VALUES ('{cocktail_id}', '{liquid_id}', '{measure}');"
        cursor.execute(sql)
    conn.commit()
    cursor.execute("SELECT * FROM Liquid_Instuctions")
    data = cursor.fetchall()
    print(data)
    conn.close()
# populate_liquid_instructions_table(liquid_ingredients_df)

In [ ]:
garnish_instructions_table_sql = "CREATE TABLE Garnish_Instuctions ( \
                                    Garnish_Instruction_ID INT NOT NULL PRIMARY KEY AUTO_INCREMENT \
                                    , Cocktail_ID INT \
                                    , Garnish_ID INT \
                                    , FOREIGN KEY (Cocktail_ID) REFERENCES Cocktails(Cocktail_ID) \
                                    , FOREIGN KEY (Garnish_ID) REFERENCES Garnishes(Garnish_ID) \
                                    );"

In [ ]:
#set empty lists for cocktail ids, liquid ingredients and measures
cocktail_ids_list = []
garnish_ingredient_list = []

#for each recipe,
for i in range(len(all_recipies)):
    #set cocktail_id
    cocktail_id = cocktail_id_list[i]
    #find ingredient list
    ingredient_list = all_recipies[i]["recipe"]
    #for each non-garnish ingredient,
    for measure, ingredient in ingredient_list:
        if measure != "add":
            pass
        else:
            cocktail_ids_list.append(cocktail_id)
            garnish_ingredient_list.append(ingredient)
            
#connect to sql
conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
#create cursor object
cursor = conn.cursor()
#use cocktail db
cursor.execute('USE cocktailproject')

#set empty list for liquid ids
garnish_ids = []

#for each ingredient,
for ingredient in garnish_ingredient_list:
    print(ingredient)
#     if ingredient[:6] == "rose's":
#         ingredient = ingredient[7:]
#         print(ingredient)
#         #select liquid id and append to liquid id list
#         sql = f'SELECT Liquid_ID FROM Liquids WHERE Liquid_Name="{ingredient}"'
#         cursor.execute(sql)
#         data = cursor.fetchall()
#         print(data[0][0])
#         liquid_ids.append(data[0][0])
#     elif ingredient[:8] =="peychaud":
#         ingredient = "peychauds bitters"
#         print(ingredient)
#         #select liquid id and append to liquid id list
#         sql = f'SELECT Liquid_ID FROM Liquids WHERE Liquid_Name="{ingredient}"'
#         cursor.execute(sql)
#         data = cursor.fetchall()
#         print(data[0][0])
#         liquid_ids.append(data[0][0])
#     else:
    #select liquid id and append to liquid id list
    sql = f'SELECT Garnish_ID FROM Garnishes WHERE Garnish_Name="{ingredient}"'
    cursor.execute(sql)
    data = cursor.fetchall()
    print(data[0][0])
    garnish_ids.append(data[0][0])

#close sql connection
conn.close()
    
garnish_ingredients_df = pd.DataFrame({"cocktail_id": cocktail_ids_list, "garnish_id": garnish_ids})

garnish_ingredients_df.head()

In [ ]:
def populate_garnish_instructions_table(garnish_instructions_df):
    conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
    cursor = conn.cursor()
    cursor.execute('USE cocktailproject')
    for row in range(len(garnish_instructions_df)):
        cocktail_id = garnish_instructions_df.iloc[row, 0]
        garnish_id = garnish_instructions_df.iloc[row, 1]
        print(garnish_id)
        sql = f"INSERT INTO Garnish_Instuctions (Cocktail_ID, Garnish_ID) VALUES ('{cocktail_id}', '{garnish_id}');"
        cursor.execute(sql)
    conn.commit()
    cursor.execute("SELECT * FROM Garnish_Instuctions")
    data = cursor.fetchall()
    print(data)
    conn.close()
populate_garnish_instructions_table(garnish_ingredients_df)

In [ ]:
rating_table_sql = "CREATE TABLE Ratings ( \
                Rating_ID INT NOT NULL PRIMARY KEY AUTO_INCREMENT \
                , Rating INT \
                , Cocktail_ID INT\
                , FOREIGN KEY (Cocktail_ID) REFERENCES Cocktails(Cocktail_ID) \
                );"

In [27]:
#connect to sql, create cursor object, and use cocktail db
conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
cursor = conn.cursor()
cursor.execute('USE cocktailproject')

sql = "SELECT Cocktail_ID FROM Cocktails"
cursor.execute(sql)
data = cursor.fetchall()
cocktail_ids_data = data
cocktail_ids = []
for i in range(len(cocktail_ids_data)):
    
    cocktail_ids.append(cocktail_ids_data[i][0])


conn.close()
len(cocktail_ids)

976

In [28]:
cocktail_id_list = []
ratings_list = []
for _id in cocktail_ids:
    for i in range(5):
        cocktail_id_list.append(_id)
        ratings_list.append(random.randint(1,5))

rating_df = pd.DataFrame({"rating": ratings_list, "cocktail_id": cocktail_id_list})

In [ ]:
def populate_rating_table(rating_df):
    conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
    cursor = conn.cursor()
    cursor.execute('USE cocktailproject')
    for row in range(len(rating_df)):
        rating = rating_df.iloc[row, 0]
        cocktail_id = rating_df.iloc[row, 1]
        print(rating)
        sql = f"INSERT INTO Ratings (Rating, Cocktail_ID) VALUES ('{rating}','{cocktail_id}');"
        cursor.execute(sql)
    conn.commit()
    cursor.execute("SELECT * FROM Ratings")
    data = cursor.fetchall()
    print(data)
    conn.close()
# populate_rating_table(rating_df)

In [32]:
all_recipies[0]

{'name': 'Gauguin',
 'category': 'Cocktail Classics',
 'instructions': 'Combine ingredients with a cup of crushed ice in blender and blend at low speed. Serve in old-fashioned glass. Top with a cherry.',
 'glass': 'Old-Fashioned Glass',
 'glass_size': 6,
 'recipe': [['2 oz', 'batavia arrack or light rum'],
  ['1 oz', 'passion fruit syrup'],
  ['1 oz', 'fresh lemon juice'],
  ['1 oz', "rose's lime juice"]]}

In [ ]:
with open('mr_boston_cocktails.txt', 'w') as text_file:
    for i in range(len(all_recipes)):
        cocktail_string = all_recipes[i]['name'] + " - "
        cocktail_string += all_recipies[i]['glass'] + " - "
        cocktail_string += all_recipies[i]["recipe"]